In [217]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback


In [218]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [219]:
load_dotenv()
api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=api_key

In [220]:
llm=HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf",model_kwargs={"temperature": 0.7, "max_length": 512})

In [221]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import PyPDF2

In [222]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above test, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the test as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [223]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [224]:
prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [260]:
chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz")

In [261]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [262]:
prompt2=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [263]:
chain2=LLMChain(llm=llm,prompt=prompt2,output_key="review")

In [264]:
generate_chain=SequentialChain(chains=[chain,chain2],input_variables=['text','number','subject','tone','response_json'],output_variables=['quiz','review'],verbose=True)

In [265]:
file_path="C:\\Users\\SHREYA SINGH\\OneDrive\\Desktop\\MCQ_Generator\\data.txt"

In [266]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [267]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.\n\nFrom a theoretical viewpoint, probably approximately correc

In [268]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [269]:
NUMBER=5
SUBJECT="machine learning"
TONE='simple'

In [270]:
response=generate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in ht


> Finished chain.


In [271]:
print(response['quiz'])


Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.

From a theoretical viewpoint, probably approximately correct

In [ ]:
import re

match = re.search(r"### YOUR_RESPONSE\s*(\{.*\})", response['quiz'], re.DOTALL)
quiz=json.loads(str(match.group(1)))

In [274]:
quiz

{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Alan Turing',
   'b': 'Donald Hebb',
   'c': 'Arthur Samuel',
   'd': 'Raytheon Company'},
  'correct': 'c'},
 '2': {'mcq': 'What is the primary goal of machine learning?',
  'options': {'a': 'To predict the future',
   'b': 'To understand human thought processes',
   'c': 'To classify data',
   'd': 'To replace human jobs'},
  'correct': 'a'},
 '3': {'mcq': 'What is the foundation of machine learning?',
  'options': {'a': 'Philosophy',
   'b': 'Statistics and mathematical optimization',
   'c': 'Artificial Intelligence',
   'd': 'Computer Science'},
  'correct': 'b'},
 '4': {'mcq': 'Which of the following is NOT a field related to machine learning?',
  'options': {'a': 'Data mining',
   'b': 'Predictive analytics',
   'c': 'Human psychology',
   'd': 'Deep learning'},
  'correct': 'c'},
 '5': {'mcq': 'What is the process of training a machine learning model called?',
  'options': {'a': 'Learning', 'b': 'Tunin

In [275]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [276]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a:Alan Turing | b:Donald Hebb | c:Arthur Samuel | d:Raytheon Company',
  'Correct': 'c'},
 {'MCQ': 'What is the primary goal of machine learning?',
  'Choices': 'a:To predict the future | b:To understand human thought processes | c:To classify data | d:To replace human jobs',
  'Correct': 'a'},
 {'MCQ': 'What is the foundation of machine learning?',
  'Choices': 'a:Philosophy | b:Statistics and mathematical optimization | c:Artificial Intelligence | d:Computer Science',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a field related to machine learning?',
  'Choices': 'a:Data mining | b:Predictive analytics | c:Human psychology | d:Deep learning',
  'Correct': 'c'},
 {'MCQ': 'What is the process of training a machine learning model called?',
  'Choices': 'a:Learning | b:Tuning | c:Training | d:Testing',
  'Correct': 'c'}]

In [277]:
quiz_df=pd.DataFrame(quiz_table_data)

In [278]:
quiz_df

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a:Alan Turing | b:Donald Hebb | c:Arthur Samue...,c
1,What is the primary goal of machine learning?,a:To predict the future | b:To understand huma...,a
2,What is the foundation of machine learning?,a:Philosophy | b:Statistics and mathematical o...,b
3,Which of the following is NOT a field related ...,a:Data mining | b:Predictive analytics | c:Hum...,c
4,What is the process of training a machine lear...,a:Learning | b:Tuning | c:Training | d:Testing,c


In [279]:
quiz_df.to_csv("Machine_learning.csv",index=False)